In [19]:
#서울열린데이터광장 인증키 상수로 저장
#깃허브에 올릴 때는 빼고 올려야 함
#! pip install python-dotenv
from dotenv import load_dotenv
import os

load_dotenv()

SERVICE_KEY =  os.getenv('SEOUL_KEY')
# print(SERVICE_KEY)

In [16]:
# https://data.seoul.go.kr/dataList/OA-21275/S/1/datasetView.do
# URL = f"http://openapi.seoul.go.kr:8088/{SERVICE_KEY}/xml/tbLnOpendataRtmsV/1/5/" #TYPE: xml
# URL = f"http://openapi.seoul.go.kr:8088/{SERVICE_KEY}/json/tbLnOpendataRtmsV/1/5/" #TYPE: json, data 개수는 5개
URL = f"http://openapi.seoul.go.kr:8088/{SERVICE_KEY}/json/tbLnOpendataRtmsV/1/1000/" #TYPE: json, data 개수는 1000개(max)
URL

'http://openapi.seoul.go.kr:8088/6f4b506873796f7534304a49575976/json/tbLnOpendataRtmsV/1/1000/'

In [17]:
import requests
import json
import pandas as pd

req = requests.get(URL)
req #<Response [200]>

<Response [200]>

In [20]:
# 데이터프레임으로 변환하기

content = req.json()
#content #'RESULT': {'CODE': 'INFO-000', 'MESSAGE': '정상 처리되었습니다'},
result = pd.DataFrame(content['tbLnOpendataRtmsV']["row"]).tail()
result

,ACC_YEAR,SGG_CD,SGG_NM,BJDONG_CD,BJDONG_NM,LAND_GBN,LAND_GBN_NM,BONBEON,BUBEON,BLDG_NM,...,OBJ_AMT,BLDG_AREA,TOT_AREA,FLOOR,RIGHT_GBN,CNTL_YMD,BUILD_YEAR,HOUSE_TYPE,REQ_GBN,RDEALER_LAWDNM
995,2024,11710,송파구,10400,송파동,1,대지,0095,0007,HANYANGLEEPS,...,15000,14.50,11.50,2.0,,,2012,연립다세대,중개거래,서울 송파구
996,2024,11560,영등포구,11000,여의도동,1,대지,0017,0009,더하우스 소호 여의도 오피스텔,...,30000,23.63,39.29,9.0,,,2017,오피스텔,중개거래,서울 마포구
997,2024,11560,영등포구,11000,여의도동,1,대지,0017,0009,더하우스 소호 여의도 오피스텔,...,29000,23.63,39.29,8.0,,,2017,오피스텔,중개거래,서울 영등포구
998,2024,11260,중랑구,10100,면목동,1,대지,0453,0037,오렌지하우스,...,22000,25.46,19.14,4.0,,,2018,연립다세대,직거래,
999,2024,11350,노원구,10400,하계동,1,대지,0271,0003,극동,...,46000,55.72,0.00,15.0,,20240117,1988,아파트,직거래,


In [21]:
URL_1 = f"http://openapi.seoul.go.kr:8088/{SERVICE_KEY}/xml/tbLnOpendataRtmsV/1/5/"

import requests
from bs4 import BeautifulSoup
import pandas as pd
import warnings
warnings.filterwarnings("ignore")


req = requests.get(URL_1)
# req.content #HTML 형태로 나옴

soup = BeautifulSoup(req.content, "lxml")
# print(soup.prettify())

# XML에서 데이터 추출하기
data = []
for item in soup.find_all("row"):
    row_data = {}
    for tag in item.find_all():
        row_data[tag.name] = tag.text
    data.append(row_data)

# 데이터 프레임 만들기
df = pd.DataFrame(data)

# 데이터프레임 표현하기
print(df)

  acc_year sgg_cd sgg_nm bjdong_cd bjdong_nm land_gbn land_gbn_nm bonbeon  \
0     2024  11560   영등포구     10200    영등포동1가        1          대지    0121   
1     2024  11740    강동구     10900       천호동        1          대지    0090   
2     2024  11320    도봉구     10800       도봉동        1          대지    0051   
3     2024  11545    금천구     10200       독산동        1          대지    1006   
4     2024  11590    동작구     10300      상도1동        1          대지    0419   

  bubeon    bldg_nm  ... obj_amt bldg_area   tot_area floor right_gbn  \
0   0005   여의도리슈빌DS  ...   20600     21.41  31.430000     7             
1   0002       도우빌라  ...   36700     48.69  46.660000    -1             
2   0012   중흥에스-클래스  ...   55000        85   0.000000     3             
3   0115  중앙하이빌1동2동  ...   23000     28.68  14.880000     4             
4   0003      종로빌리지  ...   40000     69.13  48.370000     3             

  cntl_ymd build_year house_type req_gbn rdealer_lawdnm  
0                2020       오피스텔    중개거래

In [38]:
#강사님 코드
#xml로 코드를 짜야 하는 경우

years            = soup.find_all('acc_year')         # 접수년월
sgg_cds          = soup.find_all('sgg_cd')           # 자치구코드
sgg_nms          = soup.find_all('sgg_nm')           # 자치구명
bjdong_cds       = soup.find_all('bjdong_cd')        # 법정동코드
bjdong_nms       = soup.find_all('bjdong_nm')        # 법정동명
land_gbns        = soup.find_all('land_gbn')         # 지번구분
land_gbn_nms     = soup.find_all('land_gbn_nm')      # 지번구분명
land_gbn_nms     = soup.find_all('land_gbn_nm')      # 지번구분명
bonbeons         = soup.find_all('bonbeon')          # 본번
bubeons          = soup.find_all('bubeon')           # 부번
bldg_nms         = soup.find_all('bldg_nm')          # 건물명
deal_ymds        = soup.find_all('deal_ymd')         # 계약일
obj_amts         = soup.find_all('obj_amt')          # 물건금액(만원)
bldg_areas       = soup.find_all('bldg_area')        # 건물면적(㎡)
tot_areas        = soup.find_all('tot_area')         # 토지면적(㎡)
floors           = soup.find_all('floor')            # 층
right_gbns       = soup.find_all('right_gbn')        # 권리구분
cntl_ymds        = soup.find_all('cntl_ymd')         # 취소일
build_years      = soup.find_all('build_years')      # 건축년도
house_types      = soup.find_all('house_type')       # 건물용도
req_gbn          = soup.find_all('req_gbn')          # 신고구분
rdealer_lawdnms  = soup.find_all('rdealer_lawdnm')   # 신고한 개업공인중개사 시군구명
year_list           = []
sgg_cd_list         = []
bldg_nm_list        = []
obj_amt_list        = []
house_type_list     = []
rdealer_lawdnm_list = []
for year, sgg_cd, bldg_nm, obj_amt, house_type, rdealer_lawdnm in zip(years, sgg_cds, bldg_nms, obj_amts, house_types, rdealer_lawdnms):
  year_list.append(year.get_text())
  sgg_cd_list.append(sgg_cd.get_text())
  bldg_nm_list.append(bldg_nm.get_text())
  obj_amt_list.append(obj_amt.get_text())
  house_type_list.append(house_type.get_text())
  rdealer_lawdnm_list.append(rdealer_lawdnm.get_text())
df = pd.DataFrame({
    "acc_year": year_list, 
    "sgg_cd": sgg_cd_list, 
    "bldg_nm" : bldg_nm_list,
    "obj_amt": obj_amt_list,
    "house_type" : house_type_list,
    "rdealer_lawdnm": rdealer_lawdnm_list
})
df

,acc_year,sgg_cd,bldg_nm,obj_amt,house_type,rdealer_lawdnm
0,2024,11560,여의도리슈빌DS,20600,오피스텔,서울 영등포구
1,2024,11740,도우빌라,36700,연립다세대,서울 강동구
2,2024,11320,중흥에스-클래스,55000,아파트,서울 도봉구
3,2024,11545,중앙하이빌1동2동,23000,연립다세대,
4,2024,11590,종로빌리지,40000,연립다세대,서울 강남구


In [16]:
# 공공데이터 포털
# 데이터명: 국토교통부_아파트매매 실거래자료 

PUBLIC_SERVICE_KEY = "AEvyzcHZAD3xvlLgU9yYe0DEtv0b7IXrRG4Vw6wMEx02XRjJmrrBkTwKXqAnzmHSiNzWZLnKWvRDOqhfkOGtbg%3D%3D"

In [17]:
# URL_PUBLIC = "http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTrade"

# 기술문서의 서비스 URL과 요청메시지 명세 조합

pageNo = 1 # 페이지 번호
numOfRows = 500 # 한 페이지 결과수
LAWD_CD = 11500 # 지역코드
DEAL_YMD = 202101 # 계약월
url = f'http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTradeDev?serviceKey={PUBLIC_SERVICE_KEY}&pageNo={pageNo}&numOfRows={numOfRows}&LAWD_CD={LAWD_CD}&DEAL_YMD={DEAL_YMD}'
req = requests.get(url)
req

<Response [200]>

In [19]:
req.text #xml파일로 나옴
#xml타입의 데이터를 json파일로 변경하는 방법 검색하기
# https://github.com/martinblech/xmltodict

'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultCode>99</resultCode><resultMsg>SERVICE KEY IS NOT REGISTERED ERROR.</resultMsg></header></response>'

In [14]:
# ! pip install xmltodict
# ! pip list
import xmltodict

contents = xmltodict.parse(req.text)
data = pd.DataFrame(contents["response"]["body"]["items"]["item"])
data.tail()

ExpatError: not well-formed (invalid token): line 1, column 0

In [8]:
#다중 데이터 수집하기

SERVICE_KEY = "6f4b506873796f7534304a49575976"
URL = f"http://openapi.seoul.go.kr:8088/{SERVICE_KEY}/json/tbLnOpendataRtmsV/1/1000/" #TYPE: json, data는 0에서 999까지 (max개)

In [9]:
# 내가 짠 코드 == 강사님 코드
for i in range(10):
    n = (1000 * i) + 1
    URL = f"http://openapi.seoul.go.kr:8088/{SERVICE_KEY}/json/tbLnOpendataRtmsV/{n}/{n + 999}/"
    print(URL)

http://openapi.seoul.go.kr:8088/6f4b506873796f7534304a49575976/json/tbLnOpendataRtmsV/1/1000/
http://openapi.seoul.go.kr:8088/6f4b506873796f7534304a49575976/json/tbLnOpendataRtmsV/1001/2000/
http://openapi.seoul.go.kr:8088/6f4b506873796f7534304a49575976/json/tbLnOpendataRtmsV/2001/3000/
http://openapi.seoul.go.kr:8088/6f4b506873796f7534304a49575976/json/tbLnOpendataRtmsV/3001/4000/
http://openapi.seoul.go.kr:8088/6f4b506873796f7534304a49575976/json/tbLnOpendataRtmsV/4001/5000/
http://openapi.seoul.go.kr:8088/6f4b506873796f7534304a49575976/json/tbLnOpendataRtmsV/5001/6000/
http://openapi.seoul.go.kr:8088/6f4b506873796f7534304a49575976/json/tbLnOpendataRtmsV/6001/7000/
http://openapi.seoul.go.kr:8088/6f4b506873796f7534304a49575976/json/tbLnOpendataRtmsV/7001/8000/
http://openapi.seoul.go.kr:8088/6f4b506873796f7534304a49575976/json/tbLnOpendataRtmsV/8001/9000/
http://openapi.seoul.go.kr:8088/6f4b506873796f7534304a49575976/json/tbLnOpendataRtmsV/9001/10000/


In [12]:
data_size =  3

df = None

for i in range(int(data_size)):
    n = (1000 * i) + 1
    URL = f"http://openapi.seoul.go.kr:8088/{SERVICE_KEY}/json/tbLnOpendataRtmsV/{n}/{n + 999}/"

    #하나의 URL을 json으로 받아 데이터프레임으로 만들기
    req = requests.get(URL)
    result = pd.DataFrame(content['tbLnOpendataRtmsV']["row"])
    df =  pd.concat([df, result], ignore_index=True)
    df.dropna(inplace = True)

df.tail()

,ACC_YEAR,SGG_CD,SGG_NM,BJDONG_CD,BJDONG_NM,LAND_GBN,LAND_GBN_NM,BONBEON,BUBEON,BLDG_NM,...,OBJ_AMT,BLDG_AREA,TOT_AREA,FLOOR,RIGHT_GBN,CNTL_YMD,BUILD_YEAR,HOUSE_TYPE,REQ_GBN,RDEALER_LAWDNM
2995,2024,11440,마포구,12500,성산동,1,대지,0590,0001,상암 두산위브센티움,...,16500,23.69,33.77,8.0,,,2013,오피스텔,중개거래,서울 마포구
2996,2024,11440,마포구,12500,성산동,1,대지,0592,0008,마포클레세,...,20500,20.53,33.01,8.0,,,2022,오피스텔,직거래,
2997,2024,11305,강북구,10300,수유동,1,대지,0410,0141,동아하이츠빌라,...,16300,40.32,25.47,1.0,,,1994,연립다세대,중개거래,서울 강북구
2998,2024,11560,영등포구,13200,신길동,1,대지,4955,0000,신길센트럴아이파크,...,102000,59.97,0.00,12.0,,,2019,아파트,중개거래,서울 영등포구
2999,2024,11380,은평구,11400,진관동,1,대지,0009,0000,은평지웰테라스,...,78250,84.31,97.73,2.0,,,2017,연립다세대,중개거래,서울 은평구


In [11]:
# 강사님 코드

data = None
for j in range(1,3):
    url = f'http://openapi.seoul.go.kr:8088/{SERVICE_KEY}/json/tbLnOpendataRtmsV/{1+((j-1)*1000)}/{j*1000}'
    # url = f'http://openapi.seoul.go.kr:8088/{service_key}/json/tbLnOpendataRentV/1/1000/2023/11560'
    print(url)
    req = requests.get(url)
    content = req.json()
    con = content['tbLnOpendataRtmsV']['row']
    result = pd.DataFrame(con)
    data = pd.concat([data, result])
data = data.reset_index(drop=True)
data.head()

http://openapi.seoul.go.kr:8088/6f4b506873796f7534304a49575976/json/tbLnOpendataRtmsV/1/1000
http://openapi.seoul.go.kr:8088/6f4b506873796f7534304a49575976/json/tbLnOpendataRtmsV/1001/2000


,ACC_YEAR,SGG_CD,SGG_NM,BJDONG_CD,BJDONG_NM,LAND_GBN,LAND_GBN_NM,BONBEON,BUBEON,BLDG_NM,...,OBJ_AMT,BLDG_AREA,TOT_AREA,FLOOR,RIGHT_GBN,CNTL_YMD,BUILD_YEAR,HOUSE_TYPE,REQ_GBN,RDEALER_LAWDNM
0,2024,11560,영등포구,10200,영등포동1가,1,대지,0121,0005,여의도리슈빌DS,...,20600,21.41,31.43,7.0,,,2020,오피스텔,중개거래,서울 영등포구
1,2024,11740,강동구,10900,천호동,1,대지,0090,0002,도우빌라,...,36700,48.69,46.66,-1.0,,,1991,연립다세대,중개거래,서울 강동구
2,2024,11320,도봉구,10800,도봉동,1,대지,0051,0012,중흥에스-클래스,...,55000,85.00,0.00,3.0,,,2004,아파트,중개거래,서울 도봉구
3,2024,11545,금천구,10200,독산동,1,대지,1006,0115,중앙하이빌1동2동,...,23000,28.68,14.88,4.0,,,2016,연립다세대,직거래,
4,2024,11590,동작구,10300,상도1동,1,대지,0419,0003,종로빌리지,...,40000,69.13,48.37,3.0,,,1991,연립다세대,중개거래,서울 강남구
